# Posture_Recognition
Author: Aaditya Khanal and Priyanka Pandit

In [ ]:
# label mapping using ID2Label


from datasets import load_dataset

dataset = load_dataset("C:/Users/Aaditya Khanal/OneDrive/Desktop/posture-recognition/image_dataset")

labels = dataset["train"].features["label"].names

id2label = {str(i): label for i, label in enumerate(labels)}


print(id2label)

Resolving data files:   0%|          | 0/13853 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'0': 'barbell biceps curl', '1': 'bench press', '2': 'chest fly machine', '3': 'deadlift', '4': 'decline bench press', '5': 'hammer curl', '6': 'hip thrust', '7': 'incline bench press', '8': 'lat pulldown', '9': 'lateral raises', '10': 'leg extension', '11': 'leg raises', '12': 'plank', '13': 'pull up', '14': 'push up', '15': 'romanian deadlift', '16': 'russian twist', '17': 'shoulder press', '18': 'squat', '19': 't bar row', '20': 'tricep dips', '21': 'tricep pushdown'}


In [ ]:
import gradio as gr
from transformers import AutoImageProcessor
from transformers import SiglipForImageClassification
from transformers.image_utils import load_image
from PIL import Image
import torch
import cv2
import time
import PoseModule as pm
from PIL import Image
import torch


In [4]:
import torch
print(torch.cuda.is_available())       
print(torch.cuda.get_device_name(0))    

True
NVIDIA GeForce RTX 4070 SUPER


In [5]:
model_name = "prithivMLmods/Gym-Workout-Classifier-SigLIP2"
model = SiglipForImageClassification.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name, use_fast= True)

In [17]:
def workout_classification(image):
    """Predicts workout exercise classification for an image."""
    image = Image.fromarray(image).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1).squeeze().tolist()
    
    labels = {
        "0": "barbell biceps curl", "1": "bench press", "2": "chest fly machine", "3": "deadlift",
        "4": "decline bench press", "5": "hammer curl", "6": "hip thrust", "7": "incline bench press",
        "8": "lat pulldown", "9": "lateral raises", "10": "leg extension", "11": "leg raises",
        "12": "plank", "13": "pull up", "14": "push up", "15": "romanian deadlift",
        "16": "russian twist", "17": "shoulder press", "18": "squat", "19": "t bar row",
        "20": "tricep dips", "21": "tricep pushdown"
    }
    
    predictions = {labels[str(i)]: round(probs[i], 3) for i in range(len(probs))}
    return predictions


In [ ]:
detector = pm.PoseDetector()
frame = cv2.imread("C:/Users/Aaditya Khanal/OneDrive/Desktop/posture-recognition/model-demo/src/model_demo/Bench-Press.jpg")
predictions, annotated_img = workout_classification_with_pose(frame, detector, processor, model)

# Show image with landmarks
cv2.imshow("Pose Landmarks", annotated_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Print predictions
for exercise, prob in predictions.items():
    print(f"{exercise}: {prob}")

In [13]:
iface = gr.Interface(
    fn=workout_classification,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Label(label="Prediction Scores"),
    title="Gym Workout Classification",
    description="Upload an image to classify the workout exercise."
)

In [14]:
if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
